In [1]:
import sys
assert sys.version_info >= (3, 5)
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import time
K = keras.backend
import pandas as pd
import math
import os
from sklearn.metrics import r2_score
from scipy.stats import uniform,randint,norm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.preprocessing import MinMaxScaler 

tf.random.set_seed(42)
np.random.seed(42)
random_state=42

In [2]:
save_path = 'save_path'
os.chdir(save_path)

X_train_omics_unlabelled = pd.read_csv("X_train_omics_unlabelled.csv",index_col=0)
X_train_omics_labelled = pd.read_csv("X_train_omics_labelled.csv",index_col=0)
X_test_omics= pd.read_csv("X_test_omics.csv",index_col=0)
X_valid_omics= pd.read_csv("X_valid_omics.csv",index_col=0)
features = np.load("feature_selection.npy",allow_pickle=True)

train_set_labelled_y= pd.read_csv("train_set_labelled_y.csv",index_col=0)
test_set_labelled_y= pd.read_csv("test_set_labelled_y.csv",index_col=0)
valid_set_labelled_y= pd.read_csv("valid_set_labelled_y.csv",index_col=0)

X_train_omics_unlabelled = X_train_omics_unlabelled[features]
X_train_omics_labelled = X_train_omics_labelled[features]
X_test_omics = X_test_omics[features]
X_valid_omics = X_valid_omics[features]

train_set_labelled_c= pd.read_csv("train_set_labelled_c.csv",index_col=0)
train_set_unlabelled_c= pd.read_csv("train_set_unlabelled_c.csv",index_col=0)
test_set_labelled_c= pd.read_csv("test_set_labelled_c.csv",index_col=0)
valid_set_labelled_c= pd.read_csv("valid_set_labelled_c.csv",index_col=0)


train_set_labelled_c = train_set_labelled_c[["age","male"]]
train_set_unlabelled_c = train_set_unlabelled_c[["age","male"]]
test_set_labelled_c = test_set_labelled_c[["age","male"]]
valid_set_labelled_c = valid_set_labelled_c[["age","male"]]

scaler = MinMaxScaler()
train_set_labelled_y = scaler.fit_transform(train_set_labelled_y)
valid_set_labelled_y = scaler.transform(valid_set_labelled_y)
test_set_labelled_y = scaler.transform(test_set_labelled_y)

valid_set_labelled_y[np.where(valid_set_labelled_y >1)] = 1
test_set_labelled_y[np.where(test_set_labelled_y >1)] = 1


train_set_labelled_c["age"] = scaler.fit_transform(train_set_labelled_c[["age"]])
train_set_unlabelled_c["age"] = scaler.transform(train_set_unlabelled_c[["age"]])
test_set_labelled_c["age"] = scaler.transform(test_set_labelled_c[["age"]])
valid_set_labelled_c["age"] = scaler.transform(valid_set_labelled_c[["age"]])

In [7]:
save_model_path = 'save_model_path'
os.chdir(save_model_path)


In [8]:
input_shape = X_train_omics_labelled.shape[1]

# Custom parts # 

## Helpful functions ##

In [9]:
def validation_log_lik_sampling(y_val,x_val,c_val,variational_decoder,codings_size,samples=200):

    """
    Samples a value of z for the expectation, and calculates something proportional to loglikelihood.
    
    The more samples of z, the better the MC approximation to loglik, but the longer it takes to compute.
    
    This is how we do our evaluation on the validation and also test set. 
    
    We look at the ability to generate x given y i.e. loglik(x|y,c)"""
    
    x_val_len = len(x_val)
    expectation = 0
    for i in range(samples):
        z = np.random.normal(loc=0,scale=1,size=codings_size*x_val_len).reshape(x_val_len,codings_size)
        x_pred = variational_decoder([z,y_val,c_val])
        diff = (x_val-x_pred)**2
        pdf = K.sum(diff,axis=-1)
        pdf = K.exp(-pdf)
        expectation += pdf 
    expectation = expectation / samples
    lik = tf.math.log(expectation)
    lik = K.mean(lik)    
    return lik

def create_batch(x_label, y_label, x_unlabel, c_label,c_unlabel, batch_s=32):
    '''
    Creates batches of labelled and unlabelled data. The total number of points in both batches is equal to batch_s. 
    Thanks to Omer Nivron for help with this.
    
    '''
    proportion_labelled = x_label.shape[0]/(x_label.shape[0] + x_unlabel.shape[0])
    
    shape_label = x_label.shape[0]
    label_per_batch = int(np.ceil(proportion_labelled*batch_s))
    batch_idx_la = np.random.choice(list(range(shape_label)), label_per_batch)
    batch_x_la = (x_label.iloc[batch_idx_la, :])
    batch_y_la = (y_label[batch_idx_la,:])
    batch_c_la = (c_label.iloc[batch_idx_la,:])

    
    shape_unlabel = x_unlabel.shape[0]
    unlabel_per_batch = batch_s - label_per_batch
    batch_idx_un = np.random.choice(list(range(shape_unlabel)), unlabel_per_batch)
    batch_x_un = (x_unlabel.iloc[batch_idx_un, :])
    batch_c_un = (c_unlabel.iloc[batch_idx_un,:])

    
    del batch_idx_la,batch_idx_un
            
    return batch_x_la, batch_y_la,batch_x_un,batch_c_la,batch_c_un


def progress_bar(iteration, total, size=30):
    """Progress bar for training"""
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

def print_status_bar(iteration, total, loss, metrics=None, size=30):
    """Status bar for training"""
    metrics = " - ".join(["Loss for batch: {:.4f}".format(loss)])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
def print_status_bar_epoch(iteration, total, training_loss_for_epoch,val_loss, metrics=None, size=30):
    """Status bar for training (end of epoch)"""
    metrics = " - ".join(
        ["trainLoss: {:.4f}  Val_loss: {:.4f} ".format(
            training_loss_for_epoch,val_loss)]
    )
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
    
def list_average(list_of_loss):
    return sum(list_of_loss)/len(list_of_loss)
 

def gaussian_pdf(array,mean,sigma):
    part1 = tf.math.divide(tf.constant(np.array(1.0).reshape(1,-1),dtype="float32"),sigma*(2*math.pi)**0.5)
    part2 = K.exp(-0.5*tf.math.divide((array-mean),sigma)**2)
    return part1*part2


## Model ##

### Custom components ###

In [10]:
class Sampling(keras.layers.Layer):
    """reparameterization trick"""
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var/2) + mean  
    
    
class y_dist(keras.layers.Layer):

    """
    Custom layer that is used to learn the parameters of the distribution over y.
    
    Outputs a loss. The loss is used for training. The loss is a GMM loss.
    The mean of this is then taken to provide a per batch loss. 
        
    """
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        
        
    def build(self,batch_input_shape):
        self.q1 = self.add_weight(name="q1",shape=[1,1],initializer="random_normal",trainable=True)
        self.q2 = self.add_weight(name="q2",shape=[1,1],initializer="random_normal",trainable=True)
        self.mu1 = self.add_weight(name="mu1",shape=[1,1],initializer="random_normal",trainable=True)
        self.mu2 = self.add_weight(name="mu2",shape=[1,1],initializer="random_normal",trainable=True)
        self.mu3 = self.add_weight(name="mu3",shape=[1,1],initializer="random_normal",trainable=True)
        self.tau1 = self.add_weight(name="tau1",shape=[1,1],initializer="random_normal",trainable=True)
        self.tau2 = self.add_weight(name="tau2",shape=[1,1],initializer="random_normal",trainable=True)
        self.tau3 = self.add_weight(name="tau3",shape=[1,1],initializer="random_normal",trainable=True)

        super().build(batch_input_shape)
    
    def call(self,X):
        concatenated = tf.concat([self.q1,self.q2,tf.constant(np.array(0.0).reshape(1,-1),dtype="float32")],axis=-1)
        p = K.exp(concatenated)
        p = tf.math.divide(p,K.sum(p))
        sigma_concatenated = tf.concat([self.tau1,self.tau2,self.tau3],axis=-1)
        sigma = K.exp(sigma_concatenated)
        likelihood = p[0][0]*gaussian_pdf(X,mean=self.mu1,sigma=sigma[0][0])+p[0][1]*gaussian_pdf(X,mean=self.mu2,sigma=sigma[0][1])+p[0][2]*gaussian_pdf(X,mean=self.mu3,sigma=sigma[0][2]) 
        loglik = tf.math.log(likelihood)
        loss = -loglik
        loss = tf.reduce_mean(loss)
        return loss
    
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(1)    
    
    
class FullModel_MMD(keras.models.Model):
    """
    This is the full model. For MMD. This is used for training purposes.
    
    It requires an encoder, decoder, classifier and y_distribution model to be already defined (as can be done with 
    the build_model function).
    
    It returns the nloglik i.e. the loss. 
    
    This loss can then be used in gradient descent and be minimised wrt parameters (of the four component models).
    
    At test time, you will call which of the component models you want to use (as opposed to trying to "call" this 
    FullModel which you won't want to do as its purpose is just to calculate the nloglik for training).
    
    """
    def __init__(self,N_parameter,beta,variational_encoder,variational_decoder,classifier,y_distribution,
                 codings_size,**kwargs):
        super().__init__(**kwargs)
        self.encoder = variational_encoder
        self.decoder = variational_decoder
        self.classifier = classifier  
        self.y_distribution = y_distribution
        self.codings_size = codings_size
        self.N = N_parameter
        self.beta = beta
    def call(self,inputs):
        """Inputs is a list, as such:
            inputs[0] is labelled X 
            inputs[1] is labelled y 
            inputs[2] is unlabelled X
            inputs[3] is labelled c
            inputs[4] is unlabelled c"""
        
        X_labelled = inputs[0]
        y_labelled = inputs[1]
        X_unlabelled = inputs[2]
        c_labelled = inputs[3]
        c_unlabelled = inputs[4]
        
        ############### LABELLED CASE #################
        
        codings_mean,codings_log_var,codings = self.encoder([X_labelled,y_labelled,c_labelled])
        y_pred_mean,y_pred_log_var,y_pred = self.classifier([X_labelled,c_labelled])
        reconstructions = self.decoder([codings,y_labelled,c_labelled])

        #LOSSES#
        
        recon_loss = labelled_loss_reconstruction_mmd(codings=codings,x=X_labelled,x_decoded_mean=reconstructions,
                                                      batch_size=32,codings_size=self.codings_size,beta=self.beta)
        cls_loss = labelled_cls_loss(y=y_labelled,y_pred_mean=y_pred_mean,y_pred_log_var=y_pred_log_var,N=self.N)
        y_dist_loss1 = self.y_distribution(y_labelled)
        labelled_loss = recon_loss + cls_loss + y_dist_loss1

        ############### UNLABELLED CASE #################
        y_pred_mean_unlabel,y_pred_log_var_unlabel,y_pred_unlabel = self.classifier([X_unlabelled,c_unlabelled])
        codings_mean,codings_log_var,codings = self.encoder([X_unlabelled,y_pred_unlabel,c_unlabelled])
        reconstructions_un = self.decoder([codings,y_pred_unlabel,c_unlabelled])
        
        #LOSSES#                                  
        
        unlabelled_recon_loss = unlabelled_loss_reconstruction_mmd(codings=codings,y_pred_log_var=y_pred_log_var_unlabel,
                                    y_pred_mean= y_pred_mean_unlabel,y_upper_bound=2,y_lower_bound= -1,                         
                                    x=X_unlabelled, x_decoded_mean=reconstructions_un,beta=self.beta,
                                    codings_size=self.codings_size,batch_size=32)

        y_dist_loss = self.y_distribution(y_pred_unlabel)
        unlabelled_loss = unlabelled_recon_loss + y_dist_loss
        
        ############### ALL LOSSES #######################
        
        loss = labelled_loss + unlabelled_loss
        return loss  

    

def build_model_mmd(n_hidden=1, n_neurons=723,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=300,N=30,codings_size=50):
    
    """
    Builds deep generative model.
    
    Parameters specify the architecture. Architecture is such that encoder and decoder have same number of nodes and hidden
    layers. Done for simplicity. Classifier has its own architecture.
    
    Returns encoder,decoder,y_distribution, classifier and overall model. These can be used downstream.
    
    e.g. variational_encoder,variational_decoder,classifier,y_distribution,model = build_model_mmd(n_hidden=1, n_neurons=723,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=300,N=30,codings_size=50)
    """
       
    ########## ENCODER ###############
    
    x_in = keras.layers.Input(shape=[input_shape])
    y_in = keras.layers.Input(shape=[1])
    c_in = keras.layers.Input(shape=[2])
    z = keras.layers.concatenate([x_in,y_in,c_in])
    for layer in range(n_hidden):
        z = keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="he_normal")(z)
        z = keras.layers.Dropout(0.3)(z)

    codings_mean = keras.layers.Dense(codings_size)(z)
    codings_log_var = keras.layers.Dense(codings_size)(z)
    codings = Sampling()([codings_mean, codings_log_var])
    variational_encoder = keras.models.Model(
        inputs=[x_in,y_in,c_in], outputs=[codings_mean, codings_log_var, codings])
    
    
    ########## DECODER ###############

    latent = keras.layers.Input(shape=[codings_size])
    l_merged = keras.layers.concatenate([latent,y_in,c_in])
    x = l_merged
    for layer in range(n_hidden):
        x = keras.layers.Dense(n_neurons, activation="elu",kernel_initializer="he_normal")(x)
        x = keras.layers.Dropout(0.3)(x)
    x_out = keras.layers.Dense(input_shape,activation="sigmoid")(x) 
    variational_decoder = keras.models.Model(inputs=[latent,y_in,c_in], outputs=[x_out])
    
    
    ########### CLASSIFIER ############
    
    y_classifier = keras.layers.concatenate([x_in,c_in])
    for layer in range(n_hidden_classifier):
        y_classifier = keras.layers.Dense(n_neurons_classifier, activation="elu",kernel_initializer="he_normal")(y_classifier)
        y_classifier = keras.layers.Dropout(rate=0.3)(y_classifier)
        
    y_pred_mean = keras.layers.Dense(1)(y_classifier) 
    y_pred_log_var = keras.layers.Dense(1)(y_classifier) 
    y_pred = Sampling()([y_pred_mean, y_pred_log_var])

    classifier = keras.models.Model(inputs=[x_in,c_in], outputs=[y_pred_mean,y_pred_log_var,y_pred])
    
    
    ############ Y DISTRIBUTION #############
    
    loss = y_dist()(y_in)
    y_distribution = keras.models.Model(inputs=[y_in],outputs=[loss])
    
    
    ########## FULL MODEL #############
    
    model = FullModel_MMD(N_parameter=N,beta=beta,variational_encoder=variational_encoder,
                  variational_decoder=variational_decoder,classifier=classifier,y_distribution=y_distribution,
                     codings_size=codings_size)
    
    return variational_encoder,variational_decoder,classifier,y_distribution,model

### Loss functions ###

In [11]:
def custom_mse(x,x_decoded_mean):
    """returns column of squared errors. Length of column is number of samples."""
    diff = (x-x_decoded_mean)**2
    return K.sum(diff,axis=-1) /2 

def compute_kernel(x, y):
    x_size = tf.shape(x)[0]
    y_size = tf.shape(y)[0]
    dim = tf.shape(x)[1]
    tiled_x = tf.tile(tf.reshape(x, tf.stack([x_size, 1, dim])), tf.stack([1, y_size, 1]))
    tiled_y = tf.tile(tf.reshape(y, tf.stack([1, y_size, dim])), tf.stack([x_size, 1, 1]))
    return tf.exp(-tf.reduce_mean(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float32))

def compute_mmd(x, y, sigma_sqr=1.0):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return tf.reduce_mean(x_kernel) + tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel)
    #read this for calculations: https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html
    #https://stats.stackexchange.com/questions/239008/rbf-kernel-algorithm-python
    #https://blogs.rstudio.com/ai/posts/2018-10-22-mmd-vae/
    #https://ermongroup.github.io/blog/a-tutorial-on-mmd-variational-autoencoders/

def labelled_loss_reconstruction_mmd(codings,x,x_decoded_mean,batch_size=32,codings_size=50,beta=1):
    recon_loss = custom_mse(x,x_decoded_mean)
        # Compare the generated z with true samples from a standard Gaussian, and compute their MMD distance
    true_samples = tf.random.normal(tf.stack([batch_size, codings_size]))
    loss_mmd = compute_mmd(true_samples, codings)
    return K.mean(recon_loss) + beta*loss_mmd
    

def unlabelled_loss_reconstruction_mmd(x,x_decoded_mean,codings,
                                   y_pred_log_var,y_pred_mean,y_upper_bound=2,y_lower_bound=-1,
                                  beta=1,codings_size=50,batch_size=32):
    
    """Unlabelled data. This is the reconstruction loss for the unlabelled portion. 
        The extra loss, dubbed the integral loss, is also specified here. The definition of this function follow below."""

    recon_loss = custom_mse(x, x_decoded_mean)
    true_samples = tf.random.normal(tf.stack([batch_size, codings_size]))
    loss_mmd = compute_mmd(true_samples, codings) 
    #now need to do the expectation with respect to y. Let's just use montecarlo with n=1 for now. Just like we do
    #for the expecatation with respect to z
    loss = K.mean(recon_loss) + beta*loss_mmd

    integral_loss = unlabelled_integral_loss(y_pred_log_var,y_pred_mean,y_upper_bound,y_lower_bound) 
    #returns vector of one value per sample in batch

    return K.mean(loss) + K.mean(integral_loss)

def unlabelled_integral_loss(y_pred_log_var,y_pred_mean,y_upper_bound=2, y_lower_bound=-1):
    """Unlabelled data. This is the loss resulting from the integral that arises in the loss function. I have derived
    it in my notes."""
    sigma = K.exp(y_pred_log_var/2)
    term1 = 0.5* tf.math.log(1/(2**0.5 * (math.pi)**0.5*sigma)) * tf.math.erf((y_upper_bound-y_pred_mean)/(2**0.5 *sigma))
    term2 = -0.25*tf.math.erf((y_upper_bound-y_pred_mean)/(2**0.5 *sigma))
    term3 = (y_upper_bound-y_pred_mean)*K.exp(-((y_upper_bound-y_pred_mean)**2/(2*sigma**2)))/(2**1.5 *math.pi**0.5 *sigma)
    part1 = term1 + term2 + term3
    
    term1 = 0.5* tf.math.log(1/(2**0.5 * (math.pi)**0.5*sigma)) * tf.math.erf((y_lower_bound-y_pred_mean)/(2**0.5 *sigma))
    term2 = -0.25*tf.math.erf((y_lower_bound-y_pred_mean)/(2**0.5 *sigma))
    term3 = (y_lower_bound-y_pred_mean)*K.exp(-((y_lower_bound-y_pred_mean)**2/(2*sigma**2)))/(2**1.5 *math.pi**0.5 *sigma)
    part2 = term1 + term2 + term3
    
    integral = part1-part2
    loss = integral
    return loss

def labelled_cls_loss(y,y_pred_mean,y_pred_log_var,N=383): 
    """Labelled data only. This is the loss function for the part concerning learning the distibution q(y|x). 
        This loss function depends on the mean AND the sigma, hence both are included. This can be derived from
        taking the log of the normal distribution pdf.
        
        N is the number of labelled data points in the training set. It is used with alpha to weight this loss term, so
        that the model learns q(y|x) using the labelled training data. 
    """
    alpha=0.1*N
    sigma = K.exp(y_pred_log_var/2)
    diff = (y-y_pred_mean)**2
    regression_loss = tf.math.divide(diff,(2*sigma**2))
    loss = regression_loss + 0.5*y_pred_log_var
    return alpha*K.mean(loss)

### Training functions ###

In [12]:
@tf.function
def train_step(inputs):
    """Decorated train_step function which applies a gradient update to the parameters"""
    with tf.GradientTape() as tape:
        loss = model(inputs,training=True)
        loss = tf.add_n([loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def fit_model(X_train_la, y_train_la, X_train_un,c_train_la,c_train_un,epochs,X_valid_la, y_valid_la,c_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,
             valid_set=True):

    """
    Fits a single model. Gets the validation loss too if valid set exists. 
    And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Valid set is True or False depending if you have one. If you don't, the model at the end of training is saved.
    You must still pass in dummy valid sets even if valid_set=False.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    e.g. usage fit_model(X_train_omics_labelled, train_set_labelled_y, X_train_omics_unlabelled,50,X_valid_omics, valid_set_labelled_y,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,valid_set=True)
    """
    if valid_set is True:
    
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        validation_loss = []
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0] + X_train_un.shape[0])/batch_size))

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))

                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la, batch_x_un,batch_c_la,batch_c_un= create_batch(
                        X_train_la, y_train_la, X_train_un,c_train_la,c_train_un,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la,batch_x_un.to_numpy(),batch_c_la.to_numpy(),
                             batch_c_un.to_numpy()]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0] + X_train_un.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),c_valid_la.to_numpy(),
                                                        variational_decoder=variational_decoder,codings_size=codings_size)

                validation_loss.append(val_loss)
                print_status_bar_epoch(X_train_la.shape[0] + X_train_un.shape[0]
                                 ,(X_train_la.shape[0] + X_train_un.shape[0]),training_loss_for_epoch,val_loss )

                #callback for early stopping
                if epoch <= patience - 1:

                    if epoch == 0:

                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")

                    else:
                        if all(val_loss<i for i in validation_loss[:-1]) is True:
                            variational_encoder.save("variational_encoder.h5")
                            variational_decoder.save("variational_decoder.h5")
                            classifier.save("classifier.h5")
                            y_distribution.save("y_distribution.h5")
                #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
                #then the statement below won't save any model, which is undesirable as we need to load a model. 

                if epoch > patience - 1:

                    latest_val_loss = validation_loss[-patience:]
                    if all(val_loss<i for i in latest_val_loss[:-2]) is True:
                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")
                    if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                        break     

        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5", custom_objects={
           "Sampling": Sampling
        })    
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        print("best val loss: ", min(validation_loss))
        
        return history, min(validation_loss)

        
    else:
        
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0] + X_train_un.shape[0])/batch_size))        
        val_loss = 0

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))
                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la, batch_x_un,batch_c_la,batch_c_un= create_batch(
                        X_train_la, y_train_la, X_train_un,c_train_la,c_train_un,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la,batch_x_un.to_numpy(),batch_c_la.to_numpy(),
                             batch_c_un.to_numpy()]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0] + X_train_un.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                print_status_bar_epoch(X_train_la.shape[0] + X_train_un.shape[0]
                                 ,(X_train_la.shape[0] + X_train_un.shape[0]),training_loss_for_epoch,val_loss )
        

        variational_encoder.save("variational_encoder.h5")
        variational_decoder.save("variational_decoder.h5")
        classifier.save("classifier.h5")
        y_distribution.save("y_distribution.h5")
        
        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5", custom_objects={
           "Sampling": Sampling
        })     
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        
    
        return history


def fit_model_search(X_train_la, y_train_la, X_train_un,c_train_la, c_train_un, epochs,X_valid_la, y_valid_la,c_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,
                    codings_size=50):

    """
    Use for hyperparameter search. 
    
    Fits the model. Gets the validation loss too. And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    """
    
    start = time.time()
    history = []   
       
    @tf.function
    def train_step(inputs):
        with tf.GradientTape() as tape:
            loss = model(inputs,training=True)
            loss = tf.add_n([loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
    
    validation_loss = []
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    batch_loss = []    
    batches_per_epoch = int(np.floor((X_train_la.shape[0] + X_train_un.shape[0])/batch_size))

    for epoch in range(epochs):
            
            print("Epoch {}/{}".format(epoch,epochs))
            
            for i in range(batches_per_epoch):
                
                batch_x_la, batch_y_la, batch_x_un,batch_c_la,batch_c_un= create_batch(
                    X_train_la, y_train_la, X_train_un,c_train_la,c_train_un,batch_size)

                inputs = [batch_x_la.to_numpy(),batch_y_la,batch_x_un.to_numpy(),batch_c_la.to_numpy(),
                         batch_c_un.to_numpy()]
                loss = train_step(inputs)
                batch_loss.append(loss)                
                average_batch_loss = list_average(batch_loss)                
                print_status_bar(i*batch_size,X_train_la.shape[0] + X_train_un.shape[0],average_batch_loss)
            
            training_loss_for_epoch = list_average(batch_loss)
            batch_loss = []                
            history.append(training_loss_for_epoch)            
            val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),c_valid_la.to_numpy(),
                                                    variational_decoder=variational_decoder,codings_size=codings_size)
            
            validation_loss.append(val_loss)            
            print_status_bar_epoch(X_train_la.shape[0] + X_train_un.shape[0]
                             ,(X_train_la.shape[0] + X_train_un.shape[0]),training_loss_for_epoch,val_loss )
            
            #callback for early stopping
            
            if epoch <= patience - 1:
                
                if epoch == 0:
                
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                    
                else:
                    if all(val_loss<i for i in validation_loss[:-1]) is True:
                        variational_encoder.save("variational_encoder_intermediate.h5")
                        variational_decoder.save("variational_decoder_intermediate.h5")
                        classifier.save("classifier_intermediate.h5")
                        y_distribution.save("y_distribution_intermediate.h5")
            #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
            #then the statement below won't save any model, which is undesirable as we need to load a model. 
            
            if epoch > patience - 1:
                                
                latest_val_loss = validation_loss[-patience:]
                if all(val_loss<i for i in latest_val_loss[:-1]) is True:
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                    break     
    
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder_intermediate.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder_intermediate.h5")
    classifier = keras.models.load_model("classifier_intermediate.h5", custom_objects={
           "Sampling": Sampling
        })    
    y_distribution = keras.models.load_model("y_distribution_intermediate.h5", custom_objects={
       "y_dist": y_dist
    })    
                
    done = time.time()
    elapsed = done-start
    print("Elapsed/s: ",elapsed)
    print("Final training loss: ",training_loss_for_epoch)
    print("best val loss: ", min(validation_loss))
    
    return history, min(validation_loss)

def hyperparameter_search_mmd(param_distribs,epochs,patience,n_iter,X_train_la=X_train_omics_labelled, 
                          y_train_la=train_set_labelled_y, X_train_un=X_train_omics_unlabelled,c_train_la=train_set_labelled_c,
                              c_train_un=train_set_unlabelled_c,c_valid_la=valid_set_labelled_c,
                          X_valid_la=X_valid_omics, y_valid_la=valid_set_labelled_y):
    
    """
    Performs hyperparameter, random search. Assesses performance by determining the score on the validation set. 
    
    Saves best models (encoder, decoder and regressor) and returns these. These can then be used downstream.
    
    Also returns dictionary of the search results.
    
    Param_distribs of the form: 
            param_distribs = {
            "n_hidden": [1],
            "n_hidden_classifier": [1],
            "beta": [1],
            "n_neurons": randint.rvs(50,1000-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(49,1000-49,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(50,290-50,size=30,random_state=random_state).tolist(),
            "N" :randint.rvs().tolist(),
            "learning_rate" : ....
            #"codings_size": [50]}
            
    There must be a value for every parameter. If you know the value you want to use, set it in the param_distribs
    dictionary.
    
    Patience must be less than the number of epochs.
    
    e.g. result,variational_encoder,variational_decoder,classifier,y_distribution =
            hyperparameter_search_mmd(param_distribs,500,10,n_iter=10)

            
    """

    np.random.seed(42) #needs to be here so that everything that follows is consistent

    min_val_loss = []
    master = {}

    for i in range(n_iter): 
        K.clear_session()
        master[i] = {}
        master[i]["parameters"] = {}
        
        N= np.random.choice(param_distribs["N"])
        learning_rate= np.random.choice(param_distribs["learning_rate"])
        beta= np.random.choice(param_distribs["beta"])
        n_neurons =np.random.choice(param_distribs["n_neurons"]) 
        n_neurons_classifier =np.random.choice(param_distribs["n_neurons_classifier"]) 
        n_hidden  =np.random.choice(param_distribs["n_hidden"]) 
        n_hidden_classifier  =np.random.choice(param_distribs["n_hidden_classifier"]) 
        codings_size =np.random.choice(param_distribs["codings_size"]) 
       
        master[i]["parameters"]["N"] = N
        master[i]["parameters"]["learning_rate"] = learning_rate
        master[i]["parameters"]["beta"] = beta
        master[i]["parameters"]["n_neurons"] = n_neurons
        master[i]["parameters"]["n_neurons_classifier"] = n_neurons_classifier
        master[i]["parameters"]["n_hidden"] = n_hidden
        master[i]["parameters"]["n_hidden_classifier"] = n_hidden_classifier
        master[i]["parameters"]["codings_size"] = codings_size

        
        variational_encoder,variational_decoder,classifier,y_distribution,model = build_model_mmd(n_hidden=n_hidden,       
                                       n_neurons=n_neurons,beta=beta,n_hidden_classifier=n_hidden_classifier,
                                        n_neurons_classifier=n_neurons_classifier,N=N,codings_size=codings_size)
        
                
        history,val_loss = fit_model_search(X_train_la=X_train_la, y_train_la=y_train_la, 
                                 X_train_un=X_train_un, epochs=epochs,X_valid_la=X_valid_la, 
                                 y_valid_la=y_valid_la,patience=patience,variational_encoder=variational_encoder,
                                variational_decoder=variational_decoder, classifier=classifier,
                                y_distribution=y_distribution,model=model,Sampling=Sampling,y_dist=y_dist,
                                            batch_size=32,learning_rate=learning_rate,codings_size=codings_size,
                                            
                                c_train_la=c_train_la, c_train_un = c_train_un,c_valid_la=c_valid_la
                                           )        

        master[i]["val_loss"] = val_loss
        min_val_loss.append(val_loss)

        #If val loss is lowest, save this model. 
        if val_loss <=  min(min_val_loss):
            os.rename("variational_encoder_intermediate.h5","variational_encoder.h5")
            os.rename("variational_decoder_intermediate.h5","variational_decoder.h5")
            os.rename("classifier_intermediate.h5","classifier.h5")
            os.rename("y_distribution_intermediate.h5","y_distribution.h5")

        print(master)
            
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder.h5")
    classifier = keras.models.load_model("classifier.h5", custom_objects={
           "Sampling": Sampling
        }) 
    y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
       "y_dist": y_dist
    })    

    result = sorted(master.items(), key=lambda item: item[1]["val_loss"])
    return result,variational_encoder,variational_decoder,classifier,y_distribution


# Hyperparameter Search #

In [246]:
param_distribs = {
            "n_hidden": [1,2],
            "n_hidden_classifier": [1,2],
            "beta": [1,10,15],
    #"n_neurons": [300,500],
   # "n_neurons_classifier": [50,100,150],
            "n_neurons": randint.rvs(50,600-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(20,120-20,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(20,290-20,size=30,random_state=random_state).tolist(),
   # "codings_size": [20,50,70],
            "N" :[0.1,1,10
                 
                 
            ],
            "learning_rate" : [0.001,0.0005],
            #"codings_size": [120,60]
}

In [247]:
result,variational_encoder,variational_decoder,classifier,y_distribution = hyperparameter_search_mmd(param_distribs=param_distribs,
                                                                        epochs=80,patience=10,n_iter=10)
result

Epoch 0/80

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2240/2278 [============================>.] - Loss for batch: 22.7435WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change a

2278/2278 [==============================] - trainLoss: 1.7804  Val_loss: 13.8789 
Epoch 52/80
2278/2278 [==============================] - trainLoss: 1.7665  Val_loss: 13.8356 
Elapsed/s:  75.74787616729736
Final training loss:  tf.Tensor(1.7665118, shape=(), dtype=float32)
best val loss:  tf.Tensor(13.413608, shape=(), dtype=float32)
{0: {'parameters': {'learning_rate': 0.0005, 'beta': 1, 'n_hidden': 2, 'n_neurons_classifier': 72, 'codings_size': 169, 'N': 10.0, 'n_hidden_classifier': 1, 'n_neurons': 137}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=13.031519>}, 1: {'parameters': {'learning_rate': 0.001, 'beta': 10, 'n_hidden': 2, 'n_neurons_classifier': 79, 'codings_size': 122, 'N': 1.0, 'n_hidden_classifier': 2, 'n_neurons': 508}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=13.413608>}}
Epoch 0/80

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 

2278/2278 [==============================] - trainLoss: 1.2050  Val_loss: 13.2119 
Epoch 58/80
2278/2278 [==============================] - trainLoss: 1.1641  Val_loss: 13.0905 
Epoch 59/80
2278/2278 [==============================] - trainLoss: 1.1060  Val_loss: 13.2246 
Epoch 60/80
2278/2278 [==============================] - trainLoss: 1.0547  Val_loss: 13.1358 
Epoch 61/80
2278/2278 [==============================] - trainLoss: 1.1146  Val_loss: 13.0334 
Epoch 62/80
2278/2278 [==============================] - trainLoss: 1.0770  Val_loss: 13.1367 
Epoch 63/80
2278/2278 [==============================] - trainLoss: 1.0384  Val_loss: 13.0906 
Epoch 64/80
2278/2278 [==============================] - trainLoss: 1.0224  Val_loss: 13.0382 
Epoch 65/80
2278/2278 [==============================] - trainLoss: 1.0991  Val_loss: 13.0464 
Epoch 66/80
2278/2278 [==============================] - trainLoss: 1.0961  Val_loss: 12.9150 
Epoch 67/80
2278/2278 [==============================] - train

2278/2278 [==============================] - trainLoss: 4.1408  Val_loss: 18.6233 
Epoch 7/80
2278/2278 [==============================] - trainLoss: 3.9778  Val_loss: 18.5868 
Epoch 8/80
2278/2278 [==============================] - trainLoss: 3.5854  Val_loss: 18.3307 
Epoch 9/80
2278/2278 [==============================] - trainLoss: 3.3928  Val_loss: 18.3910 
Epoch 10/80
2278/2278 [==============================] - trainLoss: 3.1453  Val_loss: 18.2849 
Epoch 11/80
2278/2278 [==============================] - trainLoss: 2.9395  Val_loss: 18.1672 
Epoch 12/80
2278/2278 [==============================] - trainLoss: 2.6681  Val_loss: 17.9881 
Epoch 13/80
2278/2278 [==============================] - trainLoss: 2.5207  Val_loss: 17.9276 
Epoch 14/80
2278/2278 [==============================] - trainLoss: 2.3713  Val_loss: 18.0301 
Epoch 15/80
2278/2278 [==============================] - trainLoss: 2.2861  Val_loss: 17.9689 
Epoch 16/80
2278/2278 [==============================] - trainLos

Epoch 0/80

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2240/2278 [============================>.] - Loss for batch: nanWARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all l

2278/2278 [==============================] - trainLoss: 1.1379  Val_loss: 13.1087 
Epoch 43/80
2278/2278 [==============================] - trainLoss: 1.2069  Val_loss: 13.2484 
Epoch 44/80
2278/2278 [==============================] - trainLoss: 1.1511  Val_loss: 13.1804 
Epoch 45/80
2278/2278 [==============================] - trainLoss: 1.1348  Val_loss: 13.1569 
Epoch 46/80
2278/2278 [==============================] - trainLoss: 1.1242  Val_loss: 13.1283 
Epoch 47/80
2278/2278 [==============================] - trainLoss: 1.1384  Val_loss: 13.0505 
Epoch 48/80
2278/2278 [==============================] - trainLoss: 1.1550  Val_loss: 13.0147 
Epoch 49/80
2278/2278 [==============================] - trainLoss: 1.0787  Val_loss: 13.0524 
Epoch 50/80
2278/2278 [==============================] - trainLoss: 1.1467  Val_loss: 13.0378 
Epoch 51/80
2278/2278 [==============================] - trainLoss: 0.9889  Val_loss: 13.0610 
Epoch 52/80
2278/2278 [==============================] - train

Epoch 0/80

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2240/2278 [============================>.] - Loss for batch: 17.9735WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change a

[(7,
  {'parameters': {'N': 10.0,
    'beta': 10,
    'codings_size': 94,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 201,
    'n_neurons_classifier': 83},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.725483>}),
 (2,
  {'parameters': {'N': 10.0,
    'beta': 10,
    'codings_size': 234,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 152,
    'n_neurons_classifier': 94},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.7303915>}),
 (8,
  {'parameters': {'N': 0.1,
    'beta': 1,
    'codings_size': 107,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 180,
    'n_neurons_classifier': 21},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.918543>}),
 (0,
  {'parameters': {'N': 10.0,
    'beta': 1,
    'codings_size': 169,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neu

Best MMD-based model has val nloglik of 12.385776. 

{'parameters': {'learning_rate': 0.001, 'beta': 15, 'n_hidden': 2, 'n_neurons_classifier': 40, 'codings_size': 171, 'N': 10.0, 'n_hidden_classifier': 1, 'n_neurons': 171}, 'val_loss': <tf.Tensor: shape=(), dtype=float32

# Single run # 

In [13]:
variational_encoder,variational_decoder,classifier,y_distribution,model = build_model_mmd(n_hidden=2, n_neurons=171,input_shape=input_shape,beta=15,n_hidden_classifier=1,
              n_neurons_classifier=40,N=10,codings_size=171)

In [14]:
fit_model(X_train_omics_labelled, train_set_labelled_y, X_train_omics_unlabelled,train_set_labelled_c,
          train_set_unlabelled_c,100,X_valid_omics, valid_set_labelled_y,valid_set_labelled_c,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,valid_set=True,codings_size=171)

Epoch 0/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2240/2278 [============================>.] - Loss for batch: 15.3301WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change 

([<tf.Tensor: shape=(), dtype=float32, numpy=15.330065>,
  <tf.Tensor: shape=(), dtype=float32, numpy=7.053376>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.9513254>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.218501>,
  <tf.Tensor: shape=(), dtype=float32, numpy=4.7441235>,
  <tf.Tensor: shape=(), dtype=float32, numpy=4.079688>,
  <tf.Tensor: shape=(), dtype=float32, numpy=3.6541464>,
  <tf.Tensor: shape=(), dtype=float32, numpy=3.2606776>,
  <tf.Tensor: shape=(), dtype=float32, numpy=3.0316384>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.8792806>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.682711>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.4193249>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.3825414>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.1199887>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.9906338>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.8242937>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.8007832>,
  <tf.Tensor: shape=(), dtype=float

In [15]:
test_nlog_lik = -validation_log_lik_sampling(test_set_labelled_y,X_test_omics.to_numpy(),test_set_labelled_c.to_numpy(),
                                    variational_decoder=variational_decoder,codings_size=171,samples=2000)
print("test_nlog_lik = " + str(test_nlog_lik))

test_nlog_lik = tf.Tensor(12.561308, shape=(), dtype=float32)


This is the best score and therefore the model that will be used.